In [29]:
import pip

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])

packages = ['PyQt6', 'nltk', 'pyvis' , 'matplotlib_inline' ,'matplotlib', 'gensim.corpora','gensim.utils','gensim.models' , 'matplotlib_inline.pyplot' ,'pandas', 'numpy', 'umap','sqlite3','spacy','win32com.client','datetime','pyvis.network','matplotlib.pyplot','plotly.graph_objects','scipy','networkx','dash','gensim','logging','warnings','nltk.corpus']


for package in packages:
    import_or_install(package)

print('done with package installation')

import nltk
import sqlite3
nltk.download('stopwords')
import pandas as pd
from pandas import Series
import numpy as np
import win32com.client
from datetime import datetime, timedelta
from dash import html
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from textblob import TextBlob
import random
import dash
import json
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash import dash_table
import dash_cytoscape as cyto

def load_outlook_messages(folder_number):
    print('loading outlook messages')
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    folder_contents = outlook.GetDefaultFolder(folder_number)
    outlook_messages = folder_contents.Items
    return outlook_messages

def outlook_message_filter(message_array, date_range):
    print('applying outlook filter')
    #filter messages processed by the last X days
    received_dt = datetime.now() - timedelta(days=date_range)
    received_dt = received_dt.strftime('%m/%d/%Y %H:%M %p')
    message_array = message_array.Restrict("[ReceivedTime] >= '" + received_dt + "'")
    return message_array

def build_core_network_arrays(messages):
    print('building core network arrays')
    #Build core tables
    pd_messages = []
    pd_recipients = []
    for message in list(messages):
        try:
            recipients_list = message.Recipients
            recipients_cleaned = ""
            for recipient in recipients_list:
                pd_recipients.append({
                    "entryID": str(message.EntryID),
                    "sender": str(message.Sender),
                    "recipient": str(recipient),
                })

            pd_messages.append({
                "entryID": str(message.EntryID),
                "conversationID": str(message.ConversationID),
                "conversationIndex": str(message.ConversationIndex),
                "createTime": str(message.CreationTime),
                "recievedTime": str(message.ReceivedTime),
                "ConversationTopic": str(message.ConversationTopic),
                "subject": str(message.Subject),
                "body": str(message.body)
            })
        except:
            print("error =  " + str(recipients_list))

    return pd_messages, pd_recipients

def core_arrays_to_pd(pd_messages, pd_recipients):
    df_messages = pd.DataFrame(pd_messages)
    df_recipients = pd.DataFrame(pd_recipients)
    return df_messages, df_recipients

def outlook_messages_to_pd(folder, days_back):
    outlook_messages = load_outlook_messages(folder)
    messages = outlook_message_filter(outlook_messages, days_back)
    pd_messages, pd_recipients = build_core_network_arrays(messages)
    df_messages, df_recipients = core_arrays_to_pd(pd_messages, pd_recipients)
    print(df_messages.shape)
    return df_messages, df_recipients

def lemmatize_with_postag(sentence):
    #Get each words Part of speech ( verb, noun etc) then pass it with the word to a lemmatizer. Then put the lemmatized word back into the Pandas DF row
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a',
                "N": 'n',
                "V": 'v',
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

def set_lowercase_and_drop_punctuation(pd_dataframe):
    #dataframe needs to have a body col which it will work on
    pd_dataframe['temp_body'] = pd_dataframe['body'].apply(lambda x: x.lower())
    pd_dataframe['temp_body'] = pd_dataframe['temp_body'].str.replace(r'[^\w\s]+', '')  #(?:\w+)
    pd_dataframe['temp_body'] = pd_dataframe['temp_body'].replace(r'\n', ' ', regex=True)
    pd_dataframe['temp_body'] = pd_dataframe['temp_body'].replace(r'\r', ' ', regex=True)
    pd_dataframe['temp_body'] = pd_dataframe['temp_body'].replace(r'_', '', regex=True)
    pd_dataframe['temp_body'] = pd_dataframe['temp_body'].str.strip()
    return pd_dataframe

def lem_and_stop_removal(pd_dataframe):
    #Remove stopwords & lemmatize
    pd_dataframe['temp_body'] = pd_dataframe['temp_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
    pd_dataframe['temp_body'] = pd_dataframe['temp_body'].apply(lemmatize_with_postag)
    return pd_dataframe

def normalize_df_col_text(pd_dataframe):
    #Text normalization bundled function
    df_messages = set_lowercase_and_drop_punctuation(pd_dataframe)
    pd_dataframe = lem_and_stop_removal(df_messages)
    return pd_dataframe

def load_core_tables_to_sqlite(db_fileName, message_table, recipient_table):
    sql_con = sqlite3.connect(db_fileName)
    message_table.to_sql("messages", sql_con, if_exists="replace")
    recipient_table.to_sql("recipients", sql_con, if_exists="replace")
    sql_con.close()

def save_to_sqlite(sqlite_file, table_to_save, tabel_name):
    sql_con = sqlite3.connect(sqlite_file)
    table_to_save.to_sql(tabel_name, sql_con, if_exists="replace")
    sql_con = sqlite3.connect(sqlite_file)
    sql_con.close()

def load_core_tables_from_sqlite(db_fileName):
    #load DF messages & recipient tables into pandas DF's to leverage
    sql_con = sqlite3.connect(db_fileName)
    message_query = 'select * from messages;'
    recipient_query = 'select recipients.* from recipients;'
    df_messages = pd.read_sql_query(message_query, sql_con)
    df_recipients = pd.read_sql_query(recipient_query, sql_con)
    df_recipients = df_recipients.drop('index', 1)
    df_messages = df_messages.drop('index', 1)
    sql_con.close()
    return df_messages, df_recipients

def create_topics(pd_messages, topic_count, tfidf, random_state):
    dtm = tfidf.fit_transform(df_messages['temp_body'])
    nmf_model = NMF(n_components=topic_count, random_state=random_state)
    nmf_model.fit(dtm)
    print("record size : ", len(tfidf.get_feature_names_out()))
    return nmf_model, dtm

def topics_to_pd(dtm, nmf_model, tfidf, topic_count):
    topics_dict={}
    for index, topic in enumerate(nmf_model.components_):
        topic_text_string =  str([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-topic_count:]])
        v = {index: str(topic_text_string)}
        topics_dict.update(v)

    topic_text_df =  pd.Series(topics_dict).to_frame()
    topic_text_df.reset_index(level=0, inplace=True)
    topic_text_df = topic_text_df.rename(columns={0: "topic_text", 'index': "topic"})
    topic_results = nmf_model.transform(dtm)
    df_messages['topic'] = topic_results.argmax(axis=1)
    return df_messages, topic_text_df

def generate_topic_colors(topics_pd):
    topic_colors = {}
    for x in range(topic_count):
        hex_color = "%06x" % random.randint(0, 0xFFFFFF)
        hex_color = "#" + hex_color
        topic_colors.update({ x: x , x : hex_color,})

    topic_color_df = pd.DataFrame(topic_colors.items(), columns=['topic','color'])
    topic_color_df = topic_color_df.merge(topic_text_df, left_on=['topic','topic'], right_on=['topic','topic'],  how='left')
    return topic_color_df

def build_network_nodes(df_nodes):
    print('building network node array')
    nodes = set()
    cy_nodes = []
    for index, row in df_nodes.iterrows():
        individual, bins = row['individual'], row['log_count']
        nodes.add(individual)
        cy_nodes.append({"data": {"id": individual, "label": individual, 'weight': bins*10, }})
    return cy_nodes

def build_network_edges(df_edges):
    print('building network edge array')
    cy_edges = []
    for index, row in df_edges.iterrows():
        source, target, topic, weight, topic_color = row['source'], row['target'], row['topic'], row['size'], row['color']
        cy_edges.append({
            'data': {
                'source': source,
                'target': target,
                'topic': topic,
                'weight': weight,
                'topic_color': topic_color
            }
        })
    return cy_edges

def generate_table(dataframe, max_rows=200):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                 html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))

        ])
    ])

#stopword definitions for later sections
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stopwords = nltk.corpus.stopwords.words('english')
nltk.download('omw-1.4')


#Set Driving Vars
topic_count = 15   #number of ML generated topics
folder_to_load =  6 #which folders to load from outlook
days_back_to_look = 100 #number of days back to pull data for

#Grab all messages from outlook folder X that came in during the lasy Y days
df_messages, df_recipients = outlook_messages_to_pd(folder_to_load, days_back_to_look)
df_messages = normalize_df_col_text(df_messages)

#save messages to sqlite
load_core_tables_to_sqlite("test_db_sqlite.db",df_messages, df_recipients)

#load messages from slite
df_messages, df_recipients = load_core_tables_from_sqlite("test_db_sqlite.db")

#Generate topics
tfidf = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', stop_words='english')
nmf_model, dtm = create_topics(df_messages, topic_count, tfidf, 42)
df_messages, topic_text_df = topics_to_pd(dtm,nmf_model, tfidf, topic_count)
topic_color_df = generate_topic_colors(topic_text_df)

#THIS CHUNK GENERATES TOPIC COLORS AND MERGES IT WITH THE MASTER TABLEs
df_messages = df_messages.merge(topic_color_df, left_on=['topic','topic'], right_on=['topic','topic'],  how='left')
df_recipients = df_messages.merge(df_recipients, left_on=['entryID','entryID'], right_on=['entryID','entryID'],  how='left')
df_recipients = df_recipients.reset_index(drop=True)

df_recipients = df_recipients.groupby(['sender',  'recipient', 'topic', 'color']).size()
df_recipients = df_recipients.to_frame(name = 'size').reset_index()
df_recipients = df_recipients.loc[df_recipients['size'] != 1]

nan_value = float("NaN")
df_recipients.replace("", nan_value, inplace=True)
df_recipients = df_recipients.dropna()

save_to_sqlite("test_db_sqlite.db",df_recipients,"compressed_data")
save_to_sqlite("test_db_sqlite.db",topic_color_df,"topic_color_df")



#Start from here to cut down on processing time
sql_con = sqlite3.connect("test_db_sqlite.db")
recipient_query = 'select * from compressed_data;'
topic_query = 'select * from topic_color_df;'

df_recipients = pd.read_sql_query(recipient_query, sql_con)
topic_color_df = pd.read_sql_query(topic_query, sql_con)

sql_con.close()

df_recipients = df_recipients.drop('index', 1)
topic_color_df = topic_color_df.drop('index', 1)


df_counts = df_recipients.groupby(['sender',  'recipient', 'topic','color']).size().reset_index().rename(columns={0: 'count'})


#node df generator
df_node_temp =  df_recipients

#build node list off of to/from table. Adding a new feature called count driven by node edge counts
df1 = df_node_temp['sender']
df2 = df_node_temp['recipient']
df_nodes = df1.append(df2)
df_nodes.reset_index()
#df_nodes.set_axis(['name'], axis=1)
df_nodes = df_nodes.to_frame()
print("Dataframe columns:", df_nodes.columns)

df_nodes = df_nodes.rename(columns={0: "individuals"})

df_nodes = df_nodes['individuals'].value_counts()

df_nodes = df_nodes.to_frame()
df_nodes = df_nodes.rename(columns={'individuals': "count"})
df_nodes.index.name = 'individual'
df_nodes.reset_index(inplace=True)



edges_table = [df_recipients["sender"], df_recipients["recipient"], df_recipients['topic'], df_recipients['color'], df_recipients['size']]
headers = ["source", "target", "topic","color", "size"]
df_edges = pd.concat(edges_table, axis=1, keys=headers)




count_mean_nodes = df_nodes['count'].mean()
count_std_dev_nodes = df_nodes['count'].std()
df_nodes['edge_count_z-score'] = (df_nodes['count'] - count_mean_nodes) / count_std_dev_nodes
values_plus_minus_one_z_score = len(df_nodes[df_nodes['edge_count_z-score'].between(-1, 1) == True])
percent_values_plus_minus_one_z_score = values_plus_minus_one_z_score / len(df_nodes) * 100
z_score_distribution_std_dev = round(df_nodes['edge_count_z-score'].std(), 2)
ser = Series(df_nodes['count'])
bins = np.quantile(np.unique(ser), np.linspace(0, 1, 11))

num_rows = np.shape(bins)[0]
bin_count = num_rows
starting_count = 5
size_labels = range(starting_count, (bin_count + starting_count) - 1, 1)

df_nodes['bins'] = pd.cut(ser, bins, include_lowest=True, labels=size_labels)

df_nodes['log_count'] = np.log(df_nodes['count'])




#THIS CHUNK SAVES TO SQLITE THE DATA PULLED FROM OUTLOOK
sql_con = sqlite3.connect("test_db_sqlite.db")
df_nodes.to_sql("df_nodes", sql_con, if_exists="replace")
df_edges.to_sql("df_edges", sql_con, if_exists="replace")
sql_con.close()



sql_con = sqlite3.connect("test_db_sqlite.db")
edges_q = 'select * from df_edges;'
nodes_q = 'select * from df_nodes;'
topics_q = 'select * from topic_color_df;'
df_edges = pd.read_sql_query(edges_q, sql_con)
df_nodes = pd.read_sql_query(nodes_q, sql_con)
topic_color_df = pd.read_sql_query(topics_q, sql_con)
sql_con.close()



df_edges = df_edges.drop('index', 1)
df_nodes = df_nodes.drop('index', 1)
topic_color_df = topic_color_df.drop('index', 1)


cyto.load_extra_layouts()
app = dash.Dash(__name__)
server = app.server



cy_nodes = build_network_nodes(df_nodes)
cy_edges = build_network_edges(df_edges)


# define stylesheet
n_stylesheet = [
    {
        "selector": 'node',  #For all nodes
        'style': {
            "opacity": 0.9,
            "height": "data(weight)",
            'width': 'data(weight)',
            "label": "data(label)",  #Label of node to display
            "background-color": "#07ABA0",  #node color
            "color": "#008B80"  #node label color
        }
    },
    {
        "selector": 'edge',  #For all edges
        "style": {
            "target-arrow-color": "#C5D3E2",  #Arrow color
            "target-arrow-shape": "triangle",  #Arrow shape
            "line-color": "data(topic_color)",  #edge color
            'arrow-scale': 2,  #Arrow size
            'curve-style': 'bezier'  #Default curve-If it is style, the arrow will not be displayed, so specify it
        }
    },
]

page_styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll',
        'min-height': '50px'
    }
}

edge_legend = topic_color_df[['topic','color']]

app.layout = html.Div(children=[
    html.H4(children='Communication patters'),

    html.Div(
        children=[
            html.Div(children=[
            cyto.Cytoscape(
                id='cytoscape',
                elements=cy_edges + cy_nodes,
                style={
                    'height': '95vh',
                    'width': '100%'
                },
                 layout={'name': 'grid'},
                stylesheet=n_stylesheet
            )], style={'width': '50%'}),
            html.Div(children=[
                    dcc.Dropdown(id='dropdown_topic',
                 options=[{'label': topic.capitalize(), 'value': topic}
                         for topic in ['0', '1', '2', '3', '4']
                         ]
                 ),

    dcc.Dropdown(
        id='dropdown-update-layout',
        options=[
            {'label': 'random',
             'value': 'random'},
            {'label': 'grid',
             'value': 'grid'},
            {'label': 'circle',
             'value': 'circle'},
            {'label': 'concentric',
             'value': 'concentric'},
            {'label': 'breadthfirst - Hiearchy',
             'value': 'breadthfirst'},
            {'label': 'klay - Force Directed',
             'value': 'klay'},
            {'label': 'cose - Force Directed',
             'value': 'cose'},
            {'label': 'cose-bilkent - Force Directed',
             'value': 'cose-bilkent'},
            {'label': 'cola - Force Directed',
             'value': 'cola'},
            {'label': 'spread - Force Directed',
             'value': 'spread'},
            {'label': 'dagre - Hiearchy',
             'value': 'dagre'}
        ], value='circle'
    ),
                html.H4(children='NodeData'),
                html.P(id='cytoscape_element_info_output'),
                html.P(id='cytoscape-tapEdgeData-output'),
                html.H4(children='Legend'),
                dash_table.DataTable(
                    data=edge_legend.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in edge_legend.columns],
                    style_cell={'textAlign': 'left'},
                    style_data_conditional=[
                        {'if': {'row_index': i, 'column_id': 'color'},
                         'background-color': edge_legend['color'][i],
                         'color': edge_legend['color'][i]} for i in range(edge_legend.shape[0])
                    ]
                     )
        ], style={'width': '50%'})
        ], style={'display': 'flex', 'flex-direction': 'row'}),

                dash_table.DataTable(
                    data=topic_color_df.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in topic_color_df.columns],
                    style_cell={'textAlign': 'left'},
                    style_data_conditional=[
                        {'if': {'row_index': i, 'column_id': 'color'},
                         'background-color': topic_color_df['color'][i],
                         'color': topic_color_df['color'][i]} for i in range(topic_color_df.shape[0])
                    ]
                     )
])

@app.callback(Output('cytoscape', 'layout'),
              Input('dropdown-update-layout', 'value'))

def update_layout(layout):
        return {
            'name': layout,
        }


@app.callback(Output('cytoscape_element_info_output', 'children'),
              Input('cytoscape', 'tapNodeData'))

def displayTapNodeData(data):
    if data:
        return "You recently clicked/tapped node: " + data['label']


@app.callback(Output('cytoscape-tapEdgeData-output', 'children'),
              Input('cytoscape', 'tapEdgeData'))

def displayTapEdgeData(data):
    if data:
        return json.dumps(data, indent=2)




if __name__ == '__main__':
    app.run_server(debug=False)

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: Could not find a version that satisfies the requirement matplotlib_inline.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib_inline.pyplot
You should consider upgrading via the 'c:\users\bensg\pycharmprojects\network_groupdetection\venv\scripts\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bensg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bensg\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


done with package installation
loading outlook messages
applying outlook filter
building core network arrays
error =  <COMObject <unknown>>
(687, 8)


C:\Users\bensg\AppData\Local\Temp\ipykernel_16144\110458729.py:116: FutureWarning:

The default value of regex will change from True to False in a future version.



KeyboardInterrupt: 